# CMU auto-graded notebook

Before you turn these assignments in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE."

---

# Building Your Own Optimizers
Optimizers tie the loss function and model parameters together by updating the model in response to the output of the loss function. In simpler terms, optimizers shape your model into its most accurate possible form by futzing with the weights. In even more simpler terms, optimizers define the way you want your model to be trained. 

The selection of optimizers could have dramatic effect on the performance of your model. If you choose a complex optimizer for a simple model, you would be likely to spend much more time in training with barely improved accuracy. And if you choose a naive optimizer for a complicated model, you would probably end up with poor accuracy.  

This exercise will cover:
- *Part 1: Calling existing optimizers provided by tf.keras*
- *Part 2: Implementing and evaluating your own optimizers*


## Part 1: Calling existing optimizers provided by tf.keras
In this part, we will build up a demo pipeline, demonstrating how to involve optimizers given by `tf.keras` into the training process. 

## (1a) Setting up the environment
Just run the following cell to establish the runtime environment. Note that we're using Tensorflow 2.0 for this part of assignment. 

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# If running locally, comment out the following try-except block. 
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import numpy as np
from tensorflow.python.ops import math_ops

## (1b) Building a baseline model
In this task, you will create a simple Linear Regression model with a synthetic toy dataset. Mean Squared Error (MSE) will be used as the loss metric. 

Note we have set a seed for `tf.random` for auto grading purpose. Please don't change the seed. 

In [2]:
# A toy dataset of points around y = 4 * x_1 + 3 * x_2 + 2
NUM_EXAMPLES = 2000
tf.random.set_seed(10605)   # DON'T CHANGE THIS!
X_train = tf.random.normal([NUM_EXAMPLES, 2])
noise = tf.random.normal([NUM_EXAMPLES, 1])
y_train = tf.matmul(X_train, [[4.], [3.]]) + 2 + noise

# Definition of the Linear Regression model
class Linear(tf.keras.Model):
      def __init__(self):
        super(Linear, self).__init__()
        self.w = tf.Variable([[5., 5.]], name='weight')
        self.b = tf.Variable(10., name='bias')

  # Implementation of Linear Regression
  # TODO: Replace <FILL IN> with appropriate code
      def call(self, inputs):
        """
          Implementation of Linear Regression. This function will use current parameters to predict the labels for inputs

          Note:
              You should try to make use of Tensorflow library to perform related calculation.

          Args:
              inputs (tf.Tensor): The input tensor of this Linear Regression model.

          Returns:
              tf.Tensor: Predicted label
        """
        return tf.matmul(inputs, self.w, transpose_b=True) + self.b

        # YOUR CODE HERE
        raise NotImplementedError()

# MSE loss function
# TODO: Replace <FILL IN> with appropriate code
def loss(y_true, y_pred):
  """
    Calculate MSE loss between the predicted label and the actual label.

    Note:
        You should try to make use of Tensorflow library to perform related calculation.

    Args:
        y_true (tf.Tensor): The actual label.
        y_pred (tf.Tensor): The predicted label.

    Returns:
        tf.Tensor: MSE loss value
  """
  return tf.math.reduce_mean((y_true-y_pred)**2)

  # YOUR CODE HERE


In [3]:
# Test for LR_MSE
m, x, y = Linear(), tf.constant([[1.0, 2.0]]), tf.constant([3.0])
assert np.equal(m(x).numpy()[0][0], 25.0), "Wrong implementation of Linear Regression"
assert np.equal(loss(m(x), y).numpy(), 484.0), "Wrong implementation of MSE loss"

## (1c) Applying an existing optimizer
Next, we will apply an Adam optimizer provided by `tf.keras` to our previously defined Linear Regression model. You can reuse this code snippet to validate your own optimizers.


In [4]:
model = Linear()
# Standard method to compile a tf.keras.Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss=loss)
# Standard method to fit a tf.keras.Model
model.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
63/63 [==============================] - 1s 1ms/step - loss: 46.4401
Epoch 2/10
63/63 [==============================] - 0s 1ms/step - loss: 4.6619
Epoch 3/10
63/63 [==============================] - 0s 1ms/step - loss: 1.0225
Epoch 4/10
63/63 [==============================] - 0s 1ms/step - loss: 1.0149
Epoch 5/10
63/63 [==============================] - 0s 1ms/step - loss: 0.9971
Epoch 6/10
63/63 [==============================] - 0s 1ms/step - loss: 0.9725
Epoch 7/10
63/63 [==============================] - 0s 1ms/step - loss: 0.9351
Epoch 8/10
63/63 [==============================] - 0s 1ms/step - loss: 0.9766
Epoch 9/10
63/63 [==============================] - 0s 1ms/step - loss: 1.0021
Epoch 10/10
63/63 [==============================] - 0s 1ms/step - loss: 0.9755


## Part 2: Implementing your own optimizers
In this part, you will implement four different optimizers by yourself! Don't be panic if you're not good at math stuff, we will give you enough instructions and explanations!

Generally, an optimizer class should contain three methods:
- `__init__`: Initializes required parameters. 
-  `calculate_gradient`: Calculates gradients in the computational graph. 
- `apply_gradient`: Updates parameters of the model. 

Some of you may notice that we're not using the "keras-style" to implement our optimizers. Since keras is a high-level encapsulation, its implementation of optimizers is generally too abstract for you to get the mathematical essence. This hacky way can enable you to design your own optimizers in a lower, Tensorflow level.

## (2a) Gradient Descent
Let's start with the simplest Gradient Descent. 

For a Linear Regression model with MSE where $\mathbf{y}=w^T\mathbf{X} + b$, the update rules of parameters at the $t$ th epoch are as follows:

$$w^{(t+1)}=w^{(t)}-\alpha \sum_{i=1}^{n} 2 \cdot (h_{w,b}(\mathbf{X}^{(i)}) - \mathbf{y}^{(i)}) \mathbf{X}^{(i)}$$

$$b^{(t+1)}=b^{(t)}-\alpha \sum_{i=1}^{n} 2 \cdot (h_{w,b}(\mathbf{X}^{(i)}) - \mathbf{y}^{(i)}))$$

where $n$ is the size of input dataset. 



In [5]:
# TODO: Replace <FILL IN> with appropriate code
class GD():
    def __init__(self, lr=0.01):
        """Initializes required parameters.

          Args:
              lr (float): Learning rate.
        """
        self.lr = lr
    def calculate_gradient(self, X, y, model):
        """Calculates gradients in the computational graph.

          Note:
              You should try to make use of Tensorflow library to perform related calculation.

          Args:
              X (tf.Tensor): The input.
              y (tf.Tensor): The actual label.
              model (tf.keras.Model): The model used to predict y with given input X. 

          Returns:
              Tuple: (gradient w.r.t w, gradient w.r.t b)
        """
        grads_w = 2.0 * tf.matmul((model(X) - y), X, transpose_a=True) * self.lr / X.shape[0]
        grads_b = 2.0 * tf.math.reduce_sum(model(X) - y) * self.lr / X.shape[0]
        return (grads_w, grads_b)

    def apply_gradient(self, grads, model):
        """Updates parameters of the model.

          Note:
              You should try to make use of Tensorflow library to perform related calculation.

          Args:
              grads (Tuple): (gradient w.r.t w, gradient w.r.t b).
              model (tf.keras.Model): The model used to predict y with given input X. 
        """
        new_w = model.w - grads[0]
        new_b = model.b - grads[1]
        model.w.assign(new_w)
        model.b.assign(new_b)

In [6]:
# Evaluation
model_0 = Linear()
opt_0 = GD()
print("Initial loss: %.3f" % (loss(model_0(X_train), y_train)))

epochs = 300
for i in range(epochs):
    # Calculate current gradients
    grads = opt_0.calculate_gradient(X_train, y_train, model_0)
    # Use the optimizer to update gradients
    opt_0.apply_gradient(grads, model_0)
    if i % 20 == 0:
        print("Loss at epoch %d: %.3f" % (i, loss(model_0(X_train), y_train)))

print("Final loss: %.3f" % loss(model_0(X_train), y_train))
print("w = %s, b = %s" % (model_0.w.numpy(), model_0.b.numpy()))

Initial loss: 68.725
Loss at epoch 0: 66.072
Loss at epoch 20: 30.250
Loss at epoch 40: 14.141
Loss at epoch 60: 6.895
Loss at epoch 80: 3.636
Loss at epoch 100: 2.169
Loss at epoch 120: 1.509
Loss at epoch 140: 1.212
Loss at epoch 160: 1.078
Loss at epoch 180: 1.018
Loss at epoch 200: 0.991
Loss at epoch 220: 0.979
Loss at epoch 240: 0.973
Loss at epoch 260: 0.971
Loss at epoch 280: 0.970
Final loss: 0.969
w = [[4.0046396 3.0552888]], b = 2.0367746


In [7]:
# Test case
assert np.allclose(loss(model_0(X_train), y_train), 0.969, atol=1e-3), 'Wrong implementation of GD'

## (2b) Stochastic Gradient Descent
Let's take one step further. In SGD, you should randomly choose one data point to calculate the gradient each time. 

In [8]:
# TODO: Replace <FILL IN> with appropriate code
class SGD():
    def __init__(self, lr=0.01):
        """Initializes required parameters.

          Args:
              lr (float): Learning rate.
        """
        self.lr = lr

    def calculate_gradient(self, X, y, model):
        """Calculates gradients in the computational graph.

          Note:
              You should try to make use of Tensorflow library to perform related calculation.

          Args:
              X (tf.Tensor): The input.
              y (tf.Tensor): The actual label.
              model (tf.keras.Model): The model used to predict y with given input X. 

          Returns:
              Tuple: (gradient w.r.t w, gradient w.r.t b)
        """
        idx = np.random.randint(0, X.shape[0])
        X_idx = tf.reshape(X[idx],(1,-1))
        y_idx = tf.reshape(y[idx],(1,-1))
        grads_w = 2.0 * (model(X_idx) - y_idx) * X_idx * self.lr
        grads_b = 2.0 * (model(X_idx) - y_idx) * self.lr
        return (grads_w, grads_b[0,0])

    def apply_gradient(self, grads, model):
        """Updates parameters of the model.

          Note:
              You should try to make use of Tensorflow library to perform related calculation.

          Args:
              grads (Tuple): (gradient w.r.t W, gradient w.r.t B).
              model (tf.keras.Model): The model used to predict y with given input X. 
        """
        new_w = model.w - grads[0]
        new_b = model.b - grads[1]
        model.w.assign(new_w)
        model.b.assign(new_b)

In [9]:
# Evaluation
model_1 = Linear()
opt_1 = SGD()

print("Initial loss: %.3f" % loss(model_1(X_train), y_train))

epochs = 300
for i in range(epochs):
  # Calculate current gradients
  grads = opt_1.calculate_gradient(X_train, y_train, model_1)
  # Use the optimizer to update gradients
  opt_1.apply_gradient(grads, model_1)
  if i % 20 == 0:
    print("Loss at epoch %d: %.3f" % (i, loss(model_1(X_train), y_train)))

print("Final loss: %.3f" % loss(model_1(X_train), y_train))
print("w = %s, b = %s" % (model_1.w.numpy(), model_1.b.numpy()))

Initial loss: 68.725
Loss at epoch 0: 66.412
Loss at epoch 20: 27.965
Loss at epoch 40: 13.937
Loss at epoch 60: 10.589
Loss at epoch 80: 5.849
Loss at epoch 100: 3.379
Loss at epoch 120: 2.316
Loss at epoch 140: 1.493
Loss at epoch 160: 1.158
Loss at epoch 180: 1.086
Loss at epoch 200: 1.030
Loss at epoch 220: 0.983
Loss at epoch 240: 0.980
Loss at epoch 260: 0.969
Loss at epoch 280: 0.982
Final loss: 0.983
w = [[3.9778802 2.965981 ]], b = 1.9353836


In [10]:
# Test case
assert np.allclose(loss(model_1(X_train), y_train), 1.0, atol=0.05), 'Wrong implementation of SGD'

## (2c) AdaGrad

One of the shortcomings about Gradient Descent is that its performance highly depends on the initial learning rate. Poor selection of initial learning rate would lead to either slow convergence or incapability of finding local minimum. In order to solve this problem, a revised version called AdaGrad is introduced. 

As its name suggests, AdaGrad makes its learning rate adaptive. To be more specific, AdaGrad uses past squared gradients to form an accumulated regularizer for its learning rate. 

Assuming $g^{(t)}$ is the gradient at $t$ th epoch, AdaGrad first calculates the accumulated squared gradients:

$$r^{(t)}=r^{(t-1)}+(g^{(t)})^2$$

Then AdaGrad will use this accumulated squared gradients to regularize its learning rate:

$$w^{(t+1)}=w^{(t)}-\frac{\alpha}{\sqrt{r^{(t)}} + \epsilon} \cdot g_w^{(t)}$$
$$b^{(t+1)}=b^{(t)}-\frac{\alpha}{\sqrt{r^{(t)}} + \epsilon} \cdot g_b^{(t)}$$

where $\epsilon$ is an additive constant (usually set as $10^{-7}$) that ensures we do not divide by 0. 

Note that you can reuse the `calculate_gradient` function in SGD for AdaGrad. 


In [11]:
# TODO: Replace <FILL IN> with appropriate code
class AdaGrad():
    def __init__(self, lr=0.001, epsilon=1e-7):
        """Initializes required parameters.

          Note: self.accumulator is the accumulated squared gradients. It is a tuple with following format:
            (accumulator w.r.t w, accumulator w.r.t b)
            And it should be initialized with the value of 0.1

          Args:
              lr (float): Learning rate.
              epsilon (float): Additive constant that ensures we do not divide by 0.
        """
        self.lr = lr
        self.epsilon = epsilon
        self.accumulator = [0.1, 0.1]

    def calculate_gradient(self, X, y, model):
        """Calculates gradients in the computational graph.

          Note:
              You should try to make use of Tensorflow library to perform related calculation.

          Args:
              X (tf.Tensor): The input.
              y (tf.Tensor): The actual label.
              model (tf.keras.Model): The model used to predict y with given input X. 

          Returns:
              Tuple: (gradient w.r.t w, gradient w.r.t b)
        """        
        idx = np.random.randint(0, X.shape[0])
        X_idx = tf.reshape(X[idx],(1,-1))
        y_idx = tf.reshape(y[idx],(1,-1))
        grads_w = 2.0 * (model(X_idx) - y_idx) * X_idx * self.lr
        grads_b = 2.0 * (model(X_idx) - y_idx) * self.lr
        
        self.accumulator[0] += grads_w ** 2
        self.accumulator[1] += grads_b[0,0] ** 2
        return (grads_w, grads_b[0,0])
    
    def apply_gradient(self, grads, model):
        """Updates parameters of the model.

          Note:
              You should try to make use of Tensorflow library to perform related calculation.

          Args:
              grads (Tuple): (gradient w.r.t w, gradient w.r.t b).
              model (tf.keras.Model): The model used to predict y with given input X. 
        """
        new_w = model.w - self.lr / (np.sqrt(self.accumulator[0])+self.epsilon) * grads[0]
        new_b = model.b - self.lr / (np.sqrt(self.accumulator[1])+self.epsilon) * grads[1]
        model.w.assign(new_w)
        model.b.assign(new_b)

In [12]:
# Evaluation
model_2 = Linear()
opt_2 = AdaGrad(lr=0.1)

print("Initial loss: %.3f" % loss(model_2(X_train), y_train))

epochs = 300
for i in range(epochs):
  # Calculate current gradients
  grads = opt_2.calculate_gradient(X_train, y_train, model_2)
  # Use the optimizer to update gradients
  opt_2.apply_gradient(grads, model_2)
  if i % 20 == 0:
    print("Loss at epoch %d: %.3f" % (i, loss(model_2(X_train), y_train)))

print("Final loss: %.3f" % loss(model_2(X_train), y_train))
print("w = %s, b = %s" % (model_2.w.numpy(), model_2.b.numpy()))

Initial loss: 68.725
Loss at epoch 0: 67.027
Loss at epoch 20: 56.407
Loss at epoch 40: 52.634
Loss at epoch 60: 48.175
Loss at epoch 80: 44.857
Loss at epoch 100: 42.364
Loss at epoch 120: 40.355
Loss at epoch 140: 38.633
Loss at epoch 160: 36.994
Loss at epoch 180: 35.458
Loss at epoch 200: 33.733
Loss at epoch 220: 32.375
Loss at epoch 240: 31.003
Loss at epoch 260: 29.761
Loss at epoch 280: 28.695
Final loss: 27.564
w = [[4.7133756 4.207177 ]], b = 7.020772


In [13]:
# Test case
assert np.allclose(loss(model_2(X_train), y_train), 29.0, atol=2.0), 'Wrong implementation of AdaGrad'


## (2d) Adam
Although it sounds like a powerful algorithm, AdaGrad still has a bunch of shortcomings. One of them is that as the accumulated squared gradients becoming larger, the learning rate will be pushed to 0, leading to an early stop. 

Adam is designed to improve AdaGrad. It relies on adaptive moment estimation to set an independent adaptive learning rate for each parameter. The idea of Adam at the $t$ th epoch is described below:
- Calculate current gradient $g^{(t)}$
- Update biased first moment estimate
  $$m^{(t)} = \beta_1\cdot m^{(t-1)}+(1-\beta_1)\cdot g^{(t)}$$
- Update biased second raw moment estimate
  $$v^{(t)} = \beta_2\cdot v^{(t-1)}+(1-\beta_2)\cdot (g^{(t)})^2$$
- Compute bias-corrected first moment estimate
  $$\hat{m}^{(t)} = \frac{m^{(t)}}{1-\beta_1^t}$$
- Compute bias-corrected second raw moment estimate
  $$\hat{v}^{(t)} = \frac{v^{(t)}}{1-\beta_2^t}$$
- Update parameters with constrained learning rate
  $$w^{(t)} = w^{(t-1)}-\alpha\cdot\frac{\hat{m}_w^{(t)}}{(\sqrt{\hat{v}_w^{(t)}}+\epsilon)}$$
  $$b^{(t)} = b^{(t-1)}-\alpha\cdot\frac{\hat{m}_b^{(t)}}{(\sqrt{\hat{v}_b^{(t)}}+\epsilon)}$$

where $\beta_1$ and $\beta_2$ are hyperparameters for the first and the second moment estimation

As we can see from the update rule, Adam dynamically forms boundaries for its learning rate using first and second moment estimate without requiring too much memory space. 

Note that you can reuse the `calculate_gradient` function in SGD for Adam. 

In [14]:
# TODO: Replace <FILL IN> with appropriate code
class Adam():
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-7):
        """Initializes required parameters.

          Note: 
            self.m is a list of Tensor, representing first moment estimations of gradients. The initial value should be [0]. 
            self.v is a list of Tensor, representing second moment estimations of gradients. The initial value should be [0]. 
            self.t is the timestep. The initial value should be 0. 

          Args:
              lr (float): Learning rate.
              epsilon (float): Additive constant that ensures we do not divide by 0.
              beta1 (float): Hyperparameter for first moment estimation
              beta2 (float): Hyperparameter for second moment estimation
        """
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.v = [0,0]
        self.m = [0,0]
        self.t = 0

    def calculate_gradient(self, X, y, model):
        """Calculates gradients in the computational graph.

          Note:
              You should try to make use of Tensorflow library to perform related calculation.

          Args:
              X (tf.Tensor): The input.
              y (tf.Tensor): The actual label.
              model (tf.keras.Model): The model used to predict y with given input X. 

          Returns:
              Tuple: (gradient w.r.t w, gradient w.r.t b)
        """
        self.t += 1
        idx = np.random.randint(0, X.shape[0])
        X_idx = tf.reshape(X[idx],(1,-1))
        y_idx = tf.reshape(y[idx],(1,-1))
        grads_w = 2.0 * (model(X_idx) - y_idx) * X_idx * self.lr
        grads_b = 2.0 * (model(X_idx) - y_idx) * self.lr
        
        self.m[0] = self.m[0] * self.beta1 + (1-self.beta1) * grads_w
        self.m[1] = self.m[1] * self.beta1 + (1-self.beta1) * grads_b[0,0]
        self.v[0] = self.v[0] * self.beta2 + (1-self.beta2) * grads_w ** 2
        self.v[1] = self.v[1] * self.beta2 + (1-self.beta2) * grads_b[0,0] ** 2

        return ([self.m[0] / (1-self.beta1 ** self.t),self.m[1] / (1-self.beta1 ** self.t)], \
                [self.v[0] / (1-self.beta2 ** self.t),self.v[1] / (1-self.beta2 ** self.t)])

    def apply_gradient(self, grads, model):
        """Updates parameters of the model.

          Note:
              You should try to make use of Tensorflow library to perform related calculation.

          Args:
              grads (Tuple): (gradient w.r.t w, gradient w.r.t b).
              model (tf.keras.Model): The model used to predict y with given input X. 
        """
        new_w = model.w - self.lr * grads[0][0] / (np.sqrt(grads[1][0])+self.epsilon)
        new_b = model.b - self.lr * grads[0][1] / (np.sqrt(grads[1][1])+self.epsilon)
        model.w.assign(new_w)
        model.b.assign(new_b)

In [15]:
# Evaluation
model_3 = Linear()
opt_3 = Adam(lr=0.1)
print("Initial loss: %.3f" % loss(model_3(X_train), y_train))

epochs = 300
for i in range(epochs):
  # Calculate current gradients
  grads = opt_3.calculate_gradient(X_train, y_train, model_3)
  # Use the optimizer to update gradients
  opt_3.apply_gradient(grads, model_3)
  if i % 20 == 0:
    print("Loss at epoch %d: %.3f" % (i, loss(model_3(X_train), y_train)))

print("Final loss: %.3f" % loss(model_3(X_train), y_train))
print("w = %s, b = %s" % (model_3.w.numpy(), model_3.b.numpy()))

Initial loss: 68.725
Loss at epoch 0: 67.329
Loss at epoch 20: 41.045
Loss at epoch 40: 22.014
Loss at epoch 60: 11.729
Loss at epoch 80: 4.431
Loss at epoch 100: 2.194
Loss at epoch 120: 1.394
Loss at epoch 140: 1.049
Loss at epoch 160: 1.016
Loss at epoch 180: 0.995
Loss at epoch 200: 1.041
Loss at epoch 220: 1.031
Loss at epoch 240: 1.059
Loss at epoch 260: 1.026
Loss at epoch 280: 0.981
Final loss: 1.215
w = [[3.7298818 2.887754 ]], b = 2.391045


In [16]:
# Test case
assert np.allclose(loss(model_3(X_train), y_train), 1.0, atol=0.4), 'Wrong implementation of Adam'
